In [ ]:
#| label: t2Fig5jn

# Prepare Python environment
import numpy as np
import scipy.io as sio
from pathlib import Path

from contextlib import contextmanager
import sys, os
from pathlib import Path

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

import os
from pathlib import Path

def find_myst_yml_directories(start_dir=None):
    """
    Recursively search for directories containing myst.yml file.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        list: List of full paths to directories containing myst.yml
    """
    if start_dir is None:
        start_dir = Path.cwd()
    else:
        start_dir = Path(start_dir)
    
    myst_dirs = []
    
    def _search_directory(current_dir):
        # Check if myst.yml exists in current directory
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            myst_dirs.append(str(current_dir.resolve()))
            # Don't search subdirectories if we found myst.yml here
            return
        
        # Recursively search all subdirectories
        for item in current_dir.iterdir():
            if item.is_dir():
                try:
                    _search_directory(item)
                except (PermissionError, OSError):
                    # Skip directories we can't access
                    continue
    
    _search_directory(start_dir)
    return myst_dirs

def find_myst_yml_directories_upwards(start_dir=None):
    """
    Search for myst.yml in current directory, if not found go to parent and repeat.
    
    Args:
        start_dir (str or Path): Starting directory (defaults to current directory)
    
    Returns:
        str or None: Full path of directory containing myst.yml, or None if not found
    """
    if start_dir is None:
        current_dir = Path.cwd()
    else:
        current_dir = Path(start_dir)
    
    # Keep going up until we reach the filesystem root
    while current_dir != current_dir.parent:  # Stop at root
        myst_file = current_dir / "myst.yml"
        if myst_file.exists():
            return str(current_dir.resolve())
        
        # Move to parent directory
        current_dir = current_dir.parent
    
    return None

with suppress_stdout():
    repo_path = Path(find_myst_yml_directories_upwards())
    print(repo_path)
    data_req_path = repo_path / "binder" / "data_requirement.json"
    data_path = repo_path / "data"
    dataset_path = data_path / "qmrlab-mooc"
    
data_dir = dataset_path / "03-T2" / "03-T2"  
data_file = "multiexpo_T2_image.mat"

#Load either archived or generated plot variables
mat_contents = sio.loadmat(data_dir / data_file)

# Get MATLAB data

# Input MET2 data with different TE values
ME_TE_1 = mat_contents['ME_TE_1']
ME_TE_2 = mat_contents['ME_TE_2']
ME_TE_3 = mat_contents['ME_TE_3']
ME_TE_4 = mat_contents['ME_TE_4']
ME_TE_4 = mat_contents['ME_TE_4']


# Output MWF, IEW and MW maps from qMRLab simulation
MWF_map = mat_contents['MWF']
T2MW_map = mat_contents['T2MW']
T2IEW_map = mat_contents['T2IEW']

## Plot
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
import numpy as np
from chart_studio.plotly import plot
from IPython.display import display, HTML
from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

config = {'showLink': False, 'displayModeBar': False}

xAxis = np.arange(0, MWF_map.shape[1])
yAxis = np.arange(0, MWF_map.shape[0])

trace1 = go.Heatmap(x=xAxis,
                    y=yAxis,
                    z=ME_TE_1,
                    colorscale='Gray',
                    showscale=False,
                    visible=False,
                    name='Signal')
trace2 = go.Heatmap(x=xAxis,
                    y=yAxis,
                    z=ME_TE_2,
                    colorscale='Gray',
                    showscale=False,
                    visible=False,
                    name='Signal')
trace3 = go.Heatmap(x=xAxis,
                    y=yAxis,
                    z=ME_TE_3,
                    colorscale='Gray',
                    showscale=False,
                    visible=True,
                    name='Signal')
trace4 = go.Heatmap(x=xAxis,
                    y=yAxis,
                    z=ME_TE_4,
                    colorscale='Gray',
                    visible=False,
                    showscale=False,
                    name='Signal')
trace5 = go.Heatmap(x=xAxis,
                    y=yAxis,
                    z=MWF_map,
                    colorscale='Portland',
                    xaxis='x2',
                    yaxis='y2',
                    visible=True,
                    name='MWF',
                    zmin=25,
                    zmax=55)
trace6 = go.Heatmap(x=xAxis,
                    y=yAxis,
                    z=T2IEW_map,
                    colorscale='Portland',
                    xaxis='x2',
                    yaxis='y2',
                    visible=False,
                    name='T2 IEW (ms)',
                    zmin=40,
                    zmax=100)
trace7 = go.Heatmap(x=xAxis,
                    y=yAxis,
                    z=T2MW_map,
                    colorscale='Portland',
                    xaxis='x2',
                    yaxis='y2',
                    visible=False,
                    name='T2 MW (ms)',
                    zmin=15,
                    zmax=40)

data = [trace1, trace2, trace3, trace4, trace5, trace6, trace7]

updatemenus = [
    dict(
        active=2,
        x=0.12,
        xanchor='left',
        y=-0.15,
        yanchor='bottom',
        direction='up',
        font=dict(family='Times New Roman', size=16),
        buttons=[
            dict(label='12.8 ms',
                 method='update',
                 args=[{'visible': [True, False, False, False, trace5['visible'], trace6['visible'], trace7['visible']]}]),
            dict(label='128 ms',
                 method='update',
                 args=[{'visible': [False, True, False, False, trace5['visible'], trace6['visible'], trace7['visible']]}]),
            dict(label='256 ms',
                 method='update',
                 args=[{'visible': [False, False, True, False, trace5['visible'], trace6['visible'], trace7['visible']]}]),
            dict(label='384 ms',
                 method='update',
                 args=[{'visible': [False, False, False, True, trace5['visible'], trace6['visible'], trace7['visible']]}])
        ],
    ),
    dict(
        x=0.7,
        xanchor='left',
        y=-0.15,
        yanchor='bottom',
        direction='up',
        font=dict(family='Times New Roman', size=16),
        buttons=[
            dict(label='MWF',
                 method='update',
                 args=[{'visible': [trace1['visible'], trace2['visible'], trace3['visible'], trace4['visible'], True, False, False]},
                       {'annotations': [
                           dict(
                               x=0.19,
                               y=1.15,
                               showarrow=False,
                               text='MET2w',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=0.85,
                               y=1.15,
                               showarrow=False,
                               text='MWF map',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=1.17,
                               y=1.15,
                               showarrow=False,
                               text='T<sub>2</sub> (ms)',
                               font=dict(family='Times New Roman', size=26),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=0.02,
                               y=-0.15,
                               showarrow=False,
                               text='TE:',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                       ]}]),
            dict(label='T2 IEW',
                 method='update',
                 args=[{'visible': [trace1['visible'], trace2['visible'], trace3['visible'], trace4['visible'], False, True, False]},
                       {'annotations': [
                           dict(
                               x=0.19,
                               y=1.15,
                               showarrow=False,
                               text='MET2w',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=0.95,
                               y=1.25,
                               showarrow=False,
                               text='T2 map for<br>Intra/Extracellular Water',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=1.17,
                               y=1.15,
                               showarrow=False,
                               text='T<sub>2</sub> (ms)',
                               font=dict(family='Times New Roman', size=26),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=0.02,
                               y=-0.15,
                               showarrow=False,
                               text='TE:',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                       ]}]),
            dict(label='T2 MW',
                 method='update',
                 args=[{'visible': [trace1['visible'], trace2['visible'], trace3['visible'], trace4['visible'], False, False, True]},
                       {'annotations': [
                           dict(
                               x=0.19,
                               y=1.15,
                               showarrow=False,
                               text='MET2w',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=0.86,
                               y=1.25,
                               showarrow=False,
                               text='T2 map for<br>Myelin Water',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=1.17,
                               y=1.15,
                               showarrow=False,
                               text='T<sub>2</sub> (ms)',
                               font=dict(family='Times New Roman', size=26),
                               xref='paper',
                               yref='paper'
                           ),
                           dict(
                               x=0.02,
                               y=-0.15,
                               showarrow=False,
                               text='TE:',
                               font=dict(family='Times New Roman', size=22),
                               xref='paper',
                               yref='paper'
                           ),
                       ]}])
        ],
    ),
]

layout = dict(
    width=728,
    height=448.5,
    margin=dict(
        t=90,
        r=140,
        b=90,
        l=30),
    annotations=[
        dict(
            x=0.19,
            y=1.15,
            showarrow=False,
            text='MET2w',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.85,
            y=1.15,
            showarrow=False,
            text='MWF map',  # Default title, will change with dropdown selection
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=1.17,
            y=1.15,
            showarrow=False,
            text='T<sub>2</sub> (ms)',
            font=dict(
                family='Times New Roman',
                size=26
            ),
            xref='paper',
            yref='paper'
        ),
        dict(
            x=0.02,
            y=-0.15,
            showarrow=False,
            text='TE:',
            font=dict(
                family='Times New Roman',
                size=22
            ),
            xref='paper',
            yref='paper'
        ),
    ],
    xaxis=dict(range=[0, xAxis[-1]], autorange=False,
               showgrid=False, zeroline=False, showticklabels=False,
               ticks='', domain=[0, 0.5]),  # First plot domain
    yaxis=dict(range=[0, yAxis[-1]], autorange=False,
               showgrid=False, zeroline=False, showticklabels=False,
               ticks='', domain=[0, 1]),
    xaxis2=dict(range=[0, xAxis[-1]], autorange=False,
                showgrid=False, zeroline=False, showticklabels=False,
                ticks='', domain=[0.5, 1]),  # Second plot domain
    yaxis2=dict(range=[0, yAxis[-1]], autorange=False,
                showgrid=False, zeroline=False, showticklabels=False,
                ticks='', domain=[0, 1], anchor='x2'),
    showlegend=False,
    autosize=False,
    updatemenus=updatemenus
)

fig = dict(data=data, layout=layout)

iplot(fig, filename='ir_fig_2.html', config=config)
